# 데이터 전처리

In [16]:
from tqdm.notebook import tqdm
import pandas as pd
import json
import glob

def load_json_files(path_pattern):
    files = glob.glob(path_pattern)
    data = []
    for file in tqdm(files, "Files"):
        with open(file, 'r', encoding='utf-8-sig') as f:
            json_data = json.load(f)
            data.append(json_data)
    return data

def create_one_hot_encoding(data):
    # 기본 데이터프레임 생성
    df = pd.DataFrame(data)
    saved = pd.DataFrame()

    # One-hot encoding을 위한 라벨 목록 정의
    labels = [
        "모욕", "욕설", "외설", "폭력위협/범죄조장", "성혐오",
        "연령", "인종/지역", "장애", "종교", "정치성향", "직업"
    ]

    # One-hot encoding 열 추가
    for label in tqdm(labels, "Labels"):
        saved[f'{label}'] = df[label].apply(lambda x: 1 if x == 'Y' else 0)
    
    # 원문 및 위치 정보 추가
    saved['문장'] = df['문장']

    return saved

# JSON 파일 로드
json_files_path = 'data/selectstar/C*.json'
data = load_json_files(json_files_path)

# 원본 데이터로부터 One-hot encoding DataFrame 생성
one_hot_df = create_one_hot_encoding(data)

# 결과 출력 및 CSV 파일로 저장(optional)
print(one_hot_df.head(n=10))
one_hot_df.to_csv('selectstar-encoded.csv', encoding='utf-8')

Files:   0%|          | 0/100000 [00:00<?, ?it/s]

Labels:   0%|          | 0/11 [00:00<?, ?it/s]

   모욕  욕설  외설  폭력위협/범죄조장  성혐오  연령  인종/지역  장애  종교  정치성향  직업  \
0   0   0   0          0    0   0      0   0   0     0   0   
1   0   0   0          0    0   0      0   0   0     0   0   
2   0   0   0          0    0   0      0   0   0     0   0   
3   0   0   0          0    0   0      0   0   0     0   0   
4   0   0   0          0    0   0      0   0   0     0   0   
5   0   0   0          0    0   0      0   0   0     0   0   
6   0   0   0          0    0   0      0   0   0     0   0   
7   0   0   0          0    0   0      0   0   0     0   0   
8   0   0   0          0    0   0      0   0   0     0   0   
9   0   0   0          0    0   0      0   0   0     0   0   

                                                  문장  
0  국민들은 깨어나야 합니다. 지금 민주당을 지지하는 사림들 보세요~ 종북주의자들이 판...  
1                      저 어른들...지금 정부 ''꼬라지''' 보는것같구만  
2  ''대깨문''' 출동 이번 코로나가 확산되는건 민노총 때문이라고 몰고가라~~~ 눼눼...  
3                            아프간과 비교를...ㅎ ''치매드셨'''나  
4  작년 국민집회때 #@이름#목사 감옥보내더니 ㅋㅋㅋ8000명 민노총은 집

In [20]:
import pandas as pd
import re

# 예시 DataFrame 생성
df = pd.read_csv("selectstar-encoded.csv", index_col=0)

# 욕설 패턴 정의 및 시작과 종료 인덱스 찾기
def find_offensive_spans(text):
    pattern = r"''(.*?)'''"
    matches = re.finditer(pattern, text)
    spans = []
    for match in matches:
        start, end = match.span()
        spans.append((start, end))
    return spans

# 각 문장에 대해 시작/종료 인덱스 추가
df['욕설 위치'] = df['문장'].apply(find_offensive_spans)
# 결과 출력
print(df[['문장', '욕설 위치']])
df.to_csv('./selectstar-location.csv', encoding="utf-8")

                                                      문장  \
0      국민들은 깨어나야 합니다. 지금 민주당을 지지하는 사림들 보세요~ 종북주의자들이 판...   
1                          저 어른들...지금 정부 ''꼬라지''' 보는것같구만   
2      ''대깨문''' 출동 이번 코로나가 확산되는건 민노총 때문이라고 몰고가라~~~ 눼눼...   
3                                아프간과 비교를...ㅎ ''치매드셨'''나   
4      작년 국민집회때 #@이름#목사 감옥보내더니 ㅋㅋㅋ8000명 민노총은 집회허락하는 이...   
...                                                  ...   
99995  \n판결 요약하면 ''기레기'''질은 맞는데 언론의 자유를 위해 봐준다임. 나쁜''...   
99996  나라를 위해 이렇게 몸 아끼지않고 헌신하는 애국자가 있는반면 파란지붕아래 ''쓰레기...   
99997  정권바뀌면 없어져야할것,,,,아무쓸모없고 25만원 받아야 아무도움안되니 세금이나 그...   
99998  그''놈'''의 #@이름#이가 뭐라고 저토록 지긋지긋하게 무죄가 만들어 주고싶어 그...   
99999  ''쌍욕달이''' ''깡패''' #@이름#과 ''망언''' ''전문''' ''문빠'...   

                                                   욕설 위치  
0                                           [(195, 204)]  
1                                             [(14, 22)]  
2                       [(0, 8), (129, 137), (229, 237)]  
3                                          

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd

# Custom dataset class
class OffensiveLanguageDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, tokenizer, max_len: int = 256):
        self.texts = dataframe['문장'].tolist()
        self.start_labels = []
        self.end_labels = []
        self.targets = dataframe['욕설'].tolist()  # 전체 욕설 여부
        self.tokenizer = tokenizer
        self.max_len = max_len

        for index, row in dataframe.iterrows():
            # 욕설 위치를 리스트로 파싱: 실제 tuple인지확인
            spans = row['욕설 위치']
            if isinstance(spans, list) and len(spans) > 0:  # 리스트가 비어있지 않다면
                try:
                    self.start_labels.append([start for start, end in spans])  # (start, end) 튜플로 분리
                    self.end_labels.append([end for start, end in spans])
                except ValueError:  # unpacking mistake
                    print(f"Error unpacking spans in row {index}: {spans}")
                    self.start_labels.append([])
                    self.end_labels.append([])
            else:
                self.start_labels.append([])
                self.end_labels.append([])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        start_label = self.start_labels[index]
        end_label = self.end_labels[index]
        target = self.targets[index]

        # Tokenize the text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        # Prepare the return dictionary
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'start_labels': torch.tensor(start_label, dtype=torch.float),  # 장면별 욕설 시작 인덱스
            'end_labels': torch.tensor(end_label, dtype=torch.float),      # 장면별 욕설 끝 인덱스
            'targets': torch.tensor(target, dtype=torch.float)              # 전체 욕설 여부
        }

class MultiOffensiveLanguageModel(nn.Module):
    def __init__(self, model_name, num_labels=1):
        super(MultiOffensiveLanguageModel, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.classification_layer = nn.Linear(self.bert.config.hidden_size, num_labels)  # 욕설 여부
        self.start_layer = nn.Linear(self.bert.config.hidden_size, 1)  # 시작 위치
        self.end_layer = nn.Linear(self.bert.config.hidden_size, 1)  # 끝 위치
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        classification_logits = self.classification_layer(hidden_states)  # [batch_size, seq_len, num_labels]
        start_logits = self.start_layer(hidden_states)  # [batch_size, seq_len, 1]
        end_logits = self.end_layer(hidden_states)  # [batch_size, seq_len, 1]
        
        return classification_logits, start_logits, end_logits

# 하이퍼파라미터 설정
max_len = 256
batch_size = 16
num_epochs = 5
learning_rate = 2e-5

# Tokenizer 및 모델 초기화
model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = MultiOffensiveLanguageModel(model_name)

# 데이터셋 및 데이터로더 준비
df = pd.read_csv('selectstar-location.csv')

dataset = OffensiveLanguageDataset(df, tokenizer, max_len)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizer 및 Loss Function 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_function_cls = nn.BCEWithLogitsLoss()  # 욕설 여부에 대한 손실 함수
loss_function_loc = nn.MSELoss()  # 시작 및 끝 위치에 대한 손실 함수

# Training loop
num_epochs = 3
model.train()

for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_labels = batch['start_labels']
        end_labels = batch['end_labels']
        targets = batch['targets']

        optimizer.zero_grad()
        classification_logits, start_logits, end_logits = model(input_ids, attention_mask)

        # 손실 계산
        # Reshape outputs to match the targets shape
        loss_cls = loss_function_cls(classification_logits.view(-1, 1), targets.unsqueeze(1).view(-1, 1))
        loss_start = loss_function_loc(start_logits, start_labels.view(-1, 1))
        loss_end = loss_function_loc(end_logits, end_labels.view(-1, 1))

        loss = loss_cls + loss_start + loss_end
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}')

# 모델 학습이 완료되었습니다.

# 모델 저장
model_save_path = './my_model'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f'Model saved at {model_save_path}')